In [ ]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt

%matplotlib inline

TRAIN_PATH = '/content/'
ipd.Audio(TRAIN_PATH + "1.wav")
ipd.Audio(TRAIN_PATH + "2.wav")
sample_rate1, audio1 = wavfile.read(TRAIN_PATH + "1.wav")
sample_rate2, audio2 = wavfile.read(TRAIN_PATH + "2.wav")

def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio
audio1 = normalize_audio(audio1)
audio2 = normalize_audio(audio2)

def frame_audio(audio, FFT_size=2048, hop_size=10, sample_rate=44100):
    # hop_size in ms
    
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1
    frames = np.zeros((frame_num,FFT_size))
    
    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]
    
    return frames
hop_size = 15 #ms
FFT_size = 2048

audio_framed1 = frame_audio(audio1, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate1)
window1 = get_window("hann", FFT_size, fftbins=True)
audio_framed2 = frame_audio(audio2, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate2)
window2 = get_window("hann", FFT_size, fftbins=True)

audio_win1 = audio_framed1 * window1
audio_win2 = audio_framed2 * window2

audio_winT1 = np.transpose(audio_win1)
audio_winT2 = np.transpose(audio_win2)

audio_fft1 = np.empty((int(1 + FFT_size // 2), audio_winT1.shape[1]), dtype=np.complex64, order='F')
audio_fft2 = np.empty((int(1 + FFT_size // 2), audio_winT2.shape[1]), dtype=np.complex64, order='F')

for n in range(audio_fft1.shape[1]):
    audio_fft1[:, n] = fft.fft(audio_winT1[:, n], axis=0)[:audio_fft1.shape[0]]
for n in range(audio_fft2.shape[1]):
    audio_fft2[:, n] = fft.fft(audio_winT2[:, n], axis=0)[:audio_fft2.shape[0]]


audio_fft1 = np.transpose(audio_fft1)
audio_fft2 = np.transpose(audio_fft2)
audio_power1 = np.square(np.abs(audio_fft1))
audio_power2 = np.square(np.abs(audio_fft2))
freq_min = 0
freq_high1 = sample_rate1 / 2
freq_high2 = sample_rate2 / 2
mel_filter_num = 10

###################################################################
def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)
    return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs

filter_points1, mel_freqs1 = get_filter_points(freq_min, freq_high1, mel_filter_num, FFT_size, sample_rate=44100)
filter_points2, mel_freqs2 = get_filter_points(freq_min, freq_high2, mel_filter_num, FFT_size, sample_rate=44100)

def get_filters(filter_points, FFT_size):
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters

filters1 = get_filters(filter_points1, FFT_size)
filters2 = get_filters(filter_points2, FFT_size)

enorm1 = 2.0 / (mel_freqs1[2:mel_filter_num+2] - mel_freqs1[:mel_filter_num])
enorm2 = 2.0 / (mel_freqs2[2:mel_filter_num+2] - mel_freqs2[:mel_filter_num])

filters1 *= enorm1[:, np.newaxis]
filters2 *= enorm2[:, np.newaxis]

audio_filtered1 = np.dot(filters1, np.transpose(audio_power1))
audio_filtered2 = np.dot(filters2, np.transpose(audio_power2))

audio_log1 = 10.0 * np.log10(audio_filtered1)
audio_log2 = 10.0 * np.log10(audio_filtered2)

def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)
    
    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
    return basis

dct_filter_num = 40

dct_filters = dct(dct_filter_num, mel_filter_num)

cepstral_coefficents1 = np.dot(dct_filters, audio_log1)
cepstral_coefficents2 = np.dot(dct_filters, audio_log2)

In [ ]:
cepstral_coefficents1[0, :]

array([-109.00630664,  -60.30016814,  -40.34842452,  -44.08375726,
        -80.66612413, -111.50989214,  -79.26212207,  -31.34617253,
        -11.95987815,   -1.55882928,    1.49996247,   -0.52053517,
         -3.37227902,  -17.17035542,  -42.00909487,  -55.98531901,
        -54.38161163,  -54.19402845,  -51.41064956,  -45.19860456,
        -34.22372848,  -26.98612758,  -27.65860594,  -27.2057359 ,
        -27.26579597,  -28.34500923,  -44.13622486,  -47.7590197 ,
        -30.78737844,  -25.09005752,  -24.55810304,  -31.3070342 ,
        -33.77693536,  -24.05346998,  -28.89098164,  -35.24966978,
        -37.89764391,  -34.71386544,  -37.02732484,  -39.97349384,
        -57.48340941,  -74.36090519,  -72.51946703,  -53.47798215,
        -40.28475772,  -41.89631141,  -60.79474633,  -94.97799072,
       -111.1464068 ,  -69.67526363,  -39.2292715 ,  -40.60485136,
        -43.1829533 ,  -51.11271359,  -67.24612461,  -76.7446699 ,
        -71.16433716,  -51.74141276,  -39.13555555,  -42.81896

In [ ]:
cepstral_coefficents2[0, :]

array([-214.60129371, -194.08662933, -149.95779457, -119.75831826,
        -88.0691576 ,  -70.04685605,  -62.96748641,  -81.61074615,
       -128.57434812, -134.08645208,  -97.22444776,  -49.43070107,
        -28.74216722,  -30.17698779,  -31.71843511,  -42.10399617,
        -72.24426299,  -75.53242934,  -70.88651534,  -74.24957729,
        -80.71358368,  -79.65359927,  -78.24334039,  -76.51231676,
        -71.95964014,  -68.07324209,  -62.10348988,  -50.29170901,
        -50.3808782 ,  -60.53048456,  -61.91987951,  -55.2948845 ,
        -46.91609565,  -39.0490794 ,  -40.28142496,  -40.86915931,
        -37.61862552,  -35.57936147,  -41.03493339,  -41.69933388,
        -49.40843276,  -48.47472399,  -53.70672963,  -72.47306978,
        -89.9657581 ,  -85.292088  ,  -58.72866305,  -44.67865345,
        -47.99318754,  -77.89476167, -108.88717355,  -98.61612672,
        -65.38742501,  -45.95405645,  -48.20429063,  -61.71393604,
        -85.64597936,  -66.71152127,  -48.11227772,  -48.24030

In [ ]:
x1, y1 = cepstral_coefficents1.shape
x2, y2 = cepstral_coefficents2.shape


In [ ]:
import numpy as np    

dim=40
random_state = np.random
H = np.eye(dim)
D = np.ones((dim,))
for n in range(1, dim):
         x = random_state.normal(size=(dim-n+1,))
         D[n-1] = np.sign(x[0])
         x[0] -= D[n-1]*np.sqrt((x*x).sum())
         Hx = (np.eye(dim-n+1) - 2.*np.outer(x, x)/(x*x).sum())
         mat = np.eye(dim)
         mat[n-1:, n-1:] = Hx
         H = np.dot(H, mat)
D[-1] = (-1)**(1-(dim % 2))*D.prod()
H = (D*H.T).T
t_H = np.transpose(H)
print(t_H)

[[-0.10897748 -0.12791779  0.14672127 ...  0.18799725  0.00192123
  -0.30441473]
 [ 0.23201351  0.11522735 -0.13995667 ... -0.05825587 -0.07998948
   0.03507464]
 [-0.18076781  0.16677235  0.0944356  ... -0.00563817  0.03802717
  -0.12605348]
 ...
 [ 0.05813642 -0.05427303 -0.00564924 ...  0.09329613 -0.29448249
  -0.00721879]
 [ 0.30932859 -0.02849347 -0.15405353 ... -0.13852661  0.28895431
  -0.23392793]
 [ 0.15784517 -0.0240524   0.2130006  ... -0.04944336 -0.01940677
   0.20316273]]


In [ ]:
Y1=np.dot(t_H,cepstral_coefficents1)
Y2=np.dot(t_H,cepstral_coefficents2)

In [ ]:
print(Y1.shape)
for i in (Y1):
  print(*i)

(40, 277)
-8.85169802109896 -13.894470439787048 -12.280077955206401 -6.7969051712486355 -0.007040843232472338 2.024727023012905 7.9988653271918375 1.5089161813571255 -6.9250417871356795 -8.345761050264707 -5.5663638918728315 0.008897683345951264 3.985725809504255 7.620351698555586 10.759409979370899 9.664505572080671 19.718415264431176 26.77162111468853 24.064449012250076 20.226807676310045 18.10121322518804 16.9057973630441 16.235169652141376 16.03924777057401 15.522934061362038 16.174165812759455 13.606240961138518 12.62202967364393 14.81506926886404 12.774839707598508 9.894301255022993 9.632212889853838 16.80009537213586 21.9908595654958 24.72411819148367 27.454587205084987 22.201173884611848 14.708184142967603 -0.861259664855257 -6.206221145230625 -4.8749163971708525 -2.4247249355689875 -4.060954693400952 -1.994920221260201 -4.063175552653212 0.7253229018720868 9.894267082281475 16.87930937386588 8.15801049456791 10.463483546993638 -0.6902963723094776 -9.20651988884564 -10.74514715

In [ ]:
import lbg 
cb_size = 32
print('generating codebook for size', cb_size)
cb, cb_abs_w, cb_rel_w = lbg.generate_codebook(Y1, cb_size)
print('output:')
for i, c in enumerate(cb):
    print('> %s, abs_weight=%d, rel_weight=%f' % (c, cb_abs_w[i], cb_rel_w[i]))

generating codebook for size 32
output:
> [-120.41096294734623, -78.62808642468201, -57.543690121853864, -56.934765716233535, -82.88012206510987, -109.08298974999724, -72.78181419036925, -37.16942705108121, -29.13218103551095, -24.44008493000857, -22.536987162449282, -23.08405128667936, -22.619914908197952, -30.54602009996741, -49.007024340363174, -59.67625637473751, -53.166430697832446, -48.904178156452915, -47.85073283721895, -48.812163644706246, -45.70569561082987, -39.97691420687892, -42.434757751533716, -40.689198875545486, -40.62028741444245, -44.758536174794656, -59.30708253993223, -62.763781915799086, -49.974153679724104, -45.162074346882164, -41.09920482419006, -43.60392030060998, -39.3636128646732, -24.844839369926586, -21.58786198127168, -25.230863847406642, -30.39302151906365, -35.59296918365318, -45.907958697806855, -51.7145595785045, -66.34302850931802, -83.06587755205481, -81.27477471589606, -65.21482630943633, -54.02262203078743, -52.585704258311786, -69.0722305870891, 

In [ ]:
m = np.array(cb)
print(m.shape)


(32, 277)


In [ ]:
lbg.avg_distortion_c_list(cb,cepstral_coefficents2)

270304.31397797517